<a href="https://colab.research.google.com/github/giovannibaratta/GTSRB/blob/master/ModelBuildStep1TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Import e definizioni { vertical-output: true, display-mode: "form" }
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab import files
import os
import time
import pprint
import math
import random as rn
import pickle
#data aug
from tensorflow import keras 
# gestione directory directory
import shutil
import sys
# trace di errori
import traceback
# colorare output
!pip install colorama
from colorama import Fore, Style

rootPath = '/gdrive/My Drive/DatasetSegnaliStradali/GTSRB/' #@param{type:'string'}

useTPU = True

print(tf.__version__)

if useTPU == False:
  print(tf.test.gpu_device_name())
else:
  if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
  else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    #with tf.Session(tpu_address) as session:
    #  devices = session.list_devices()

    #print('TPU devices:')
    #pprint.pprint(devices)
      
tf.logging.set_verbosity(tf.logging.ERROR)
drive.mount('/gdrive')

#import script da gdrive
scriptPath = rootPath + "scripts/colab/"
sys.path.append(scriptPath)
from CommonUtils import *
from ModelBuilderUtils import *
from TrainingUtils import *

resetSeed()

In [0]:
#@title Caricamento dei dati { vertical-output: true, display-mode: "form" }

#riproducibilità dei training
resetSeed()
    
trainingDir = rootPath + 'data/training/'
validationDir = rootPath + 'data/validation/'

width = 48#@param{type:'integer'}
height = 48#@param{type:'integer'}

useCustomLoad = False #@param {type:"boolean"}
balanceClass = False #@param {type:"boolean"}
customRatio = { 
                  0 : 1.3,
                  1 : 0.5,
                  2 : 0.5,
                  3 : 0.7,
                  4 : 0.5,
                  5 : 0.5,
                  6 : 1,
                  7 : 0.7,
                  8 : 0.7,
                  9 : 0.7,
                  10 : 0.5,
                  11 : 0.7,
                  12 : 0.5,
                  13 : 0.5,
                  14 : 1,
                  15 : 1,
                  16 : 1,
                  17 : 1,
                  18 : 0.7,
                  19 : 1.3,
                  20 : 1,
                  21 : 1,
                  22 : 1,
                  23 : 1,
                  24 : 1.2,
                  25 : 0.5,
                  26 : 0.7,
                  27 : 1.2,
                  28 : 1,
                  29 : 1,
                  30 : 1,
                  31 : 1,
                  32 : 1.2,
                  33 : 1,
                  34 : 1,
                  35 : 0.7,
                  36 : 1,
                  37 : 1.2,
                  38 : 0.5,
                  39 : 1,
                  40 : 1,
                  41 : 1,
                  42 : 1
              }

useDataAugmentation = False #@param {type:"boolean"}
NUMBER_OF_TRAINING_AUG = 3 #@param {type:"slider", min:0, max:20, step:1}
NUMBER_OF_VALIDATION_AUG = 1 #@param {type:"slider", min:0, max:5, step:1}

generatorCustomLoad = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=5,
    width_shift_range=3,
    height_shift_range=3,
    shear_range=0.1,
    zoom_range=(0.95,1.05),
    channel_shift_range=0.01)

generatorTraining = keras.preprocessing.image.ImageDataGenerator(
    brightness_range = (0.25,0.6),
    rotation_range=25,
    width_shift_range=8,
    height_shift_range=8,
    shear_range=15,
    zoom_range=(0.4,1.15),
    channel_shift_range=0.13)

generatorValidation = keras.preprocessing.image.ImageDataGenerator(
    brightness_range = (0.25,0.7),
    rotation_range=25,
    width_shift_range=8,
    height_shift_range=8,
    shear_range=0.4,
    zoom_range=(0.4,1.15),
    channel_shift_range=0.12)

if useCustomLoad == False:
  for key in customRatio.keys():
    customRatio[key] = 1.0
    
if useCustomLoad == True and balanceClass == True:
  trClassCount = {}
  files = list(filter(lambda fn : fn.startswith('resized'), os.listdir(trainingDir)))
  for fileName in files:
    classLabel = int(fileName.replace("resized",""))
    numFilePath = trainingDir + 'num' + str(classLabel)
    with open(numFilePath, 'r') as file:
      originalNum = int(file.readline())
      trClassCount[classLabel] = originalNum

  newMin = min(trClassCount.values()) * 1.3
  for key in trClassCount.keys():
    val =  trClassCount[key]
    if val < newMin:
      customRatio[key] = max(newMin/val, 1.3)
    elif val == newMin:
      customRatio[key] = 1.0
    elif val > newMin:
      if val > newMin/2:
        customRatio[key] = max(newMin/val, 0.4)
      else:
        customRatio[key] = max(newMin/val, 0.5)

# numero di immagini originali disponibili per ogni classe
trainingClassCount = {}
validationClassCount = {}
totalTrainingCount = 0

#leggo quante img ci sono per ogni class di training 
files = list(filter(lambda fn : fn.startswith('resized'), os.listdir(trainingDir)))
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  numFilePath = trainingDir + 'num' + str(classLabel)
  with open(numFilePath, 'r') as file:
    originalNum = int(file.readline())
    trainingClassCount[classLabel] = originalNum
    totalTrainingCount += round(originalNum * customRatio[classLabel])

if useDataAugmentation == True:
  originaltotalTrainingCount = totalTrainingCount
  totalTrainingCount = totalTrainingCount * (NUMBER_OF_TRAINING_AUG + 1)
    
trainingPadEnd = False #inserire altre immagini alla fine per essere divisibile per 8

print("Training images (no pad):", totalTrainingCount)

if totalTrainingCount % 8 != 0:
  # se uso le TPU la lunghezza dei dati deve essere divisibile per 8
  trainingToPad = 8 - (totalTrainingCount % 8)
  totalTrainingCount = totalTrainingCount + (8 - (totalTrainingCount % 8))
  trainingPadEnd = True
  
#preparo gli array che contengono le img di training
trainingData = np.empty((totalTrainingCount, width, height, 3), dtype='float32')
trainingLabels = np.empty((totalTrainingCount), dtype='uint8')

#leggo quante img ci sono per ogni class di validation
validationNumber = 0

for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  numFilePath = validationDir + 'num' + str(classLabel)
  with open(numFilePath, 'r') as file:
    numFile = int(file.readline())
    validationClassCount[classLabel] = numFile
    validationNumber += numFile

validationPadEnd = False #inserire altre immagini alla fine per essere divisibile per 8

if useDataAugmentation == True:
  originalValidationNumber = validationNumber
  validationNumber = validationNumber * (NUMBER_OF_VALIDATION_AUG + 1)

print("Validation images (no pad):", validationNumber)

if validationNumber % 8 != 0:
  # se uso le TPU la lunghezza dei dati deve essere divisibile per 8
  validationToPad = 8 - (validationNumber % 8)
  validationNumber = validationNumber + (8 - (validationNumber % 8))
  validationPadEnd = True
  
#preparo gli array che contengono le img di validation
validationData = np.empty((validationNumber, width, height, 3), dtype='float32')
validationLabels = np.empty((validationNumber), dtype='uint8')

print('Total', validationNumber + totalTrainingCount)
if useDataAugmentation == True:
  print("Data augmentation training", NUMBER_OF_TRAINING_AUG, "validation", NUMBER_OF_VALIDATION_AUG)
else:
  print("Data augmentation : OFF")
  
validationCount = 0
trainingCount = 0
loadedClass = 0

#caricamento dati
for fileName in files:
  classLabel = int(fileName.replace("resized",""))
  #carico le img di training originali
  trainingImages = np.memmap(trainingDir + fileName, dtype=np.uint8,
              mode='r', shape=(trainingClassCount[classLabel],width,height,3))

  imagesToLoad = round(trainingClassCount[classLabel] * customRatio[classLabel])
  #genero delle immagini aggiuntive se custom load è > 1.0
  imageToGenerate = max(0, imagesToLoad - trainingClassCount[classLabel])
  indices = generateIndices(imageToGenerate, trainingClassCount[classLabel])
  for index in indices:
    trainingData[trainingCount] = generateRandomImage(generatorCustomLoad, trainingImages[index])/255.0
    trainingLabels[trainingCount] = classLabel
    trainingCount += 1
  imagesToLoad = imagesToLoad - imageToGenerate
  #numero di img aggiunte per la classe
  indices = generateIndices(imagesToLoad, trainingClassCount[classLabel])
  for index in indices:
    trainingData[trainingCount] = trainingImages[index]/255.0
    trainingLabels[trainingCount] = classLabel
    trainingCount += 1
  #carico le immagini di validation
  validationImages = np.memmap(validationDir + fileName, dtype=np.uint8,
              mode='r', shape=(validationClassCount[classLabel],width,height,3))

  for img in validationImages:
    validationData[validationCount] = img/255.0
    validationLabels[validationCount] = classLabel
    validationCount += 1

  loadedClass += 1
  if (loadedClass) % 5 == 0 or loadedClass == len(files):
    print('Caricamento file ', loadedClass, 'su', len(files))

# fine for caricamento classi
# per ogni immagine genero delle nuove immagini con trasformazioni casuali
if useDataAugmentation == True:
  print("Genero le immagini aggiuntive")
  resetSeed(10)
  for imageIndex in range(0, trainingCount):
    for augIndex in range(0, NUMBER_OF_TRAINING_AUG):
      trainingData[trainingCount] = generateRandomImage(generatorTraining, trainingData[imageIndex])/255.0
      trainingLabels[trainingCount] = trainingLabels[imageIndex]
      trainingCount += 1

  resetSeed(-50)
  for imageIndex in range(0, validationCount):
    for augIndex in range(0, NUMBER_OF_VALIDATION_AUG):
      validationData[validationCount] = generateRandomImage(generatorValidation, validationData[imageIndex])/255.0
      validationLabels[validationCount] = validationLabels[imageIndex]
      validationCount += 1
  
if trainingPadEnd == True:
  for index in range(1, trainingToPad + 1):
    trainingData[-index] = trainingData[len(trainingData) - (trainingToPad + 1)]
    trainingLabels[-index] = trainingLabels[len(trainingLabels) - (trainingToPad + 1)]

if validationPadEnd == True:
  for index in range(1, validationToPad + 1):
    validationData[-index] = validationData[len(validationData) - (validationToPad + 1)]
    validationLabels[-index] = validationLabels[len(validationLabels) - (validationToPad + 1)]

print("")

print('Training', trainingData.shape) 
print('Validation', validationData.shape)

plt.figure(figsize=(20,5))
plt.hist(trainingLabels, bins=43)
plt.xticks(np.arange(0, 42, 1))
plt.xlim(left=0, right=42)
plt.show()

In [0]:
# parametri training
trInfo = TrainingInfo.getDefaultTPU(trainingData,
                                    trainingLabels,
                                    validationData,
                                    validationLabels)

trInfo.setParameters(
    learningRateList = [1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
    fineTuningIterations = 0,
    mainEpochs = 50, 
    metrics = ['sparse_categorical_accuracy']
)

models = Models()
inputs = tf.keras.layers.Input(shape=(width, height, 3))
###
# DEFINIZIONE MODELLI
###

layer = buildDenseLayer(inputs, layers = 1, size = 64, regularizers = 0.01, flattenInput = True)
outputs = buildDenseSoftmax(layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
models.addModel('1D64', model, trInfo)

layer = buildDenseLayer(inputs, layers = 2, size = 64, regularizers = 0.01, dropout = 0.3,flattenInput = True)
outputs = buildDenseSoftmax(layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
models.addModel('2D64', model, trInfo)


layer = buildDenseLayer(inputs, layers = 1, size = 128, regularizers = 0.01,flattenInput = True)
outputs = buildDenseSoftmax(layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
models.addModel('1D128', model, trInfo)

layer = buildDenseLayer(inputs, layers = 2, size = 128, regularizers = 0.01, dropout = 0.3,flattenInput = True)
outputs = buildDenseSoftmax(layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
models.addModel('2D128', model, trInfo)

layer = buildConvLayer(inputs, layers = 1, size = 64, kernelSize = 3, flatten = True)
outputs = buildDenseSoftmax(layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
models.addModel('1C64_3', model, trInfo)


layer = buildConvLayer(inputs, layers = 2, size = 64, kernelSize = 3, flatten = True)
outputs = buildDenseSoftmax(layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
models.addModel('2C64_3', model, trInfo)

layer = buildConvLayer(inputs, layers = 2, size = 64, kernelSize = 3, flatten = True)
layer =  buildDenseLayer(layer, layers = 1, size = 64, regularizers = 0.01, flattenInput = False)
outputs = buildDenseSoftmax(layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
models.addModel('2C64_3_1D64', model, trInfo)

layer = buildConvBlock(inputs, layers = 2, size = 64, kernelSize = 3, poolSize = 2, poolStrides = 2, flatten = False)
layer =  buildDenseLayer(layer, layers = 1, size = 64, regularizers = 0.01, flattenInput = True)
outputs = buildDenseSoftmax(layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
models.addModel('2C64_3_1MAX2_2_1D64', model, trInfo)


layer = buildConvBlock(inputs, layers = 3, size = 64, kernelSize = 3, poolSize = 2, poolStrides = 2, flatten = False)
layer = buildInceptionBlock(layer)
outputs = buildGlobalSoftmax(layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
models.addModel('3C64_3_1I_G', model, trInfo)


layer = buildConvBlock(inputs, layers = 3, size = 64, kernelSize = 3, poolSize = 2, poolStrides = 2, flatten = False)
layer = buildResNetBlock(layer, 128, 128, 512)
outputs = buildGlobalSoftmax(layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
models.addModel('3C64_3_1R_G', model, trInfo)

In [0]:
#@title Training { vertical-output: true, display-mode: "form" }
cleanLastSession = False #@param {type:"boolean"}
if 'lastSession' in locals():
  if cleanLastSession == True:
    for trainedModel in lastSession:
      print("Rimuovo il training del modello", trainedModel)
      shutil.rmtree(trainedModel)
    lastSession = []
else:
  lastSession = []
#@markdown ---
verboseTraining = 1 #@param {type:"integer"}
#@markdown ---
mainTrainingEarlyStoppingDelta = 0.1 #@param {type:"number"}
mainTrainingEarlyStoppinPatience = 10 #@param {type:"integer"}
fineTuningEarlyStoppingDelta = 0.005 #@param {type:"number"}
fineTuningEarlyStoppinPatience = 7 #@param {type:"integer"}
#@markdown ---

trainingPath = rootPath + 'trainingPhase1/'

###
# TRAINING
###

for modelName in models.getModelsName():
  localtime = time.localtime(time.time())
  model = models.getModel(modelName)
  trainingInfo = models.getModelTrainingInfo(modelName)
  lossFunction = trainingInfo.getLossFunction()
  metrics = trainingInfo.getMetrics()
  resumeTraining = trainingInfo.getResumeTraining()
  #Prepare la directory principale training/modelNameAndtime/
  skipModelDefinition = False
  learningListToResume = []
  if resumeTraining != None:
    modelPath = trainingPath + modelName + "_" + resumeTraining[0] + "/"
    skipModelDefinition = True
    learningListToResume = [info[0] for info in resumeTraining[1]]
  else:
    modelPath = trainingPath + modelName + "_" + buildTimestampName(localtime) + "/"
  
  try:
    lastSession.append(modelPath)
    tensorboardPath = modelPath + "tensorboard/"
    if skipModelDefinition == False:
      os.mkdir(modelPath)
      with open(modelPath + 'model.json', 'w') as modelFile:
        jsonModel = model.to_json()
        modelFile.write(jsonModel)
      with open(modelPath + "trainingParameters.txt","w") as trainingLog:
        trainingLog.write("AUG : "+ str(useDataAugmentation)+"\n")
        trainingLog.write("TR_AUG : " + str(NUMBER_OF_TRAINING_AUG)+"\n")
        trainingLog.write("VAL_AUG : " + str(NUMBER_OF_VALIDATION_AUG)+"\n")
        trainingLog.write("Custom_load : " + str(useCustomLoad)+"\n")
        if useCustomLoad == True:
          trainingLog.write("Custom_ratio : " + str(customRatio) + "\n")
        freezeLayer = trainingInfo.getFreezeLayer()
        trainingLog.write("freezeLayer : " + str(freezeLayer)+ "\n")
        if freezeLayer == True:
          trainingLog.write("freezeFrom : " + str(trainingInfo.getFreezeFrom())+ "\n")
          trainingLog.write("freezeIndex : " + str(trainingInfo.getLayersToFreeze())+ "\n")
    else:
      with open(modelPath + "resumedTrainingParameters.txt","w") as trainingLog:
        trainingLog.write("AUG : "+ str(useDataAugmentation)+"\n")
        trainingLog.write("TR_AUG : " + str(NUMBER_OF_TRAINING_AUG)+"\n")
        trainingLog.write("VAL_AUG : " + str(NUMBER_OF_VALIDATION_AUG)+"\n")
        trainingLog.write("Custom_load : " + str(useCustomLoad)+"\n")
        if useCustomLoad == True:
          trainingLog.write("Custom_ratio : " + str(customRatio) + "\n")
        freezeLayer = trainingInfo.getFreezeLayer()
        trainingLog.write("freezeLayer : " + str(freezeLayer)+ "\n")
        if freezeLayer == True:
          trainingLog.write("freezeFrom : " + str(trainingInfo.getFreezeFrom())+ "\n")
          trainingLog.write("freezeIndex : " + str(trainingInfo.getLayersToFreeze())+ "\n")
    for learningRate in trainingInfo.getLearningRateList():
      if useTPU == True:
        # se non si resetta la prima iterazione impiega sempre più tempo
        tf.reset_default_graph() 
      print(Style.BRIGHT, "\n" + modelName + "] : main training con lr" + str(learningRate), Style.RESET_ALL)
      learningPath = modelPath + str(learningRate) + "/" #training/modelNameAndTime/learningRate/
      mainTrainingPath = learningPath + "Main/" #training/modelNameAndTime/learningRate/Main/
      weightsPath = mainTrainingPath + "weights/" #training/modelNameAndTime/learningRate/Main/weights/
        
      if learningRate not in learningListToResume:
        # Preparo le direcotry 
        plotsPath = mainTrainingPath + "plots/" #training/modelNameAndTime/learningRate/Main/plots/
        historyPath = mainTrainingPath + "history/" #training/modelNameAndTime/learningRate/Main/history/
        makeDirs([learningPath, mainTrainingPath, weightsPath, plotsPath, historyPath])
        # main training
        tensorboardIterationPath = tensorboardPath + "MainTraining_" + str(learningRate) + "/"
        checkpointPath = '/tmp/epoch_{epoch:02d}_valLoss_{val_loss:.4f}.hdf5'
        #checkpointPath = weightsPath + "epoch_{epoch:02d}_valLoss_{val_loss:.4f}.hdf5"
        callbacks = [
            tensorboardCallback(logDir = tensorboardIterationPath,saveGraph = True, saveImages = True),
            earlyStoppingCallback(mainTrainingEarlyStoppingDelta , mainTrainingEarlyStoppinPatience ),
            checkpointCallback(fileName = checkpointPath, bestOnly = True, weightsOnly = True)
        ]
        weights = trainingInfo.getWeights()
        weightsFromFile = trainingInfo.getWeightsFromFile()
        
        history, weights = trainModel(
            model, learningRate,
            trainingInfo.getTrainingData(), trainingInfo.getTrainingLabels(),
            trainingInfo.getValidationData(), trainingInfo.getValidationLabels(),
            trainingInfo.getMainEpochs(), trainingInfo.getShuffle(), trainingInfo.getBatchSize(),
            lossFunction, metrics,
            callbacks = callbacks, weights = weights,weightsFromFile = weightsFromFile, verbose = verboseTraining)
        cleanCheckpointCallbackWeights('/tmp/')
        lastWeights = list(filter(lambda fn: fn.startswith('epoch') and fn.endswith('.hdf5'),os.listdir('/tmp/')))[0]
        shutil.move('/tmp/' + lastWeights, weightsPath + lastWeights)
        #cleanCheckpointCallbackWeights(weightsPath)
        saveLossPlot(plotsPath + "validationLoss.png", "Validation",[(history.history['val_loss'], learningRate)])
        saveLossPlot(plotsPath + "traininLoss.png", "Training",[(history.history['loss'], learningRate)])
        if 'val_sparse_categorical_accuracy' in history.history:
          saveAccuracyPlot(plotsPath + "validationAccuracy.png", "Validation",[(history.history['val_sparse_categorical_accuracy'],learningRate)])
        if 'sparse_categorical_accuracy' in history.history:
          saveAccuracyPlot(plotsPath + "trainingAccuracy.png", "Training",[(history.history['sparse_categorical_accuracy'],learningRate)])
        with open(historyPath + "history.history","wb") as pickleOut:
          pickle.dump(history.history, pickleOut)
      else:
        weights = retrieveWeights(model, weightsPath) 
      # fine-tuning
      if trainingInfo.getFineTuningIterations() > 0:
        fineTuningPath = learningPath + "FineTuning/" #training/modelNameAndTime/learningRate/FineTuning/
        makeDirs([fineTuningPath])
        if learningRate not in learningListToResume:
          learningRateUpdater = trainingInfo.getLearningRateUpdater()
        else:
          learningRateUpdater = list(filter(lambda lrAndUpdater:lrAndUpdater[0] == learningRate,resumeTraining[1]))[0][1]
        for iteration in range(trainingInfo.getFineTuningIterations()):
          if useTPU == True:
            # se non si resetta la prima iterazione impiega sempre più tempo
            tf.reset_default_graph()
          fineTuningLR = learningRateUpdater(learningRate, iteration + 1)
          print(Style.BRIGHT, "\n" + modelName + "] : fine tuning con lr" + str(fineTuningLR) + "partendo da" + str(learningRate), Style.RESET_ALL)
          #preparo le directory
          fineTuningLRPath = fineTuningPath + str(fineTuningLR) + "/" #training/modelNameAndTime/learningRate/FineTuning/LearningRate/
          weightsPath = fineTuningLRPath + "weights/" #training/modelNameAndTime/learningRate/FineTuning/LearningRate/weights/
          plotsPath = fineTuningLRPath + "plots/" #training/modelNameAndTime/learningRate/FineTuning/LearningRate/plots/
          historyPath = fineTuningLRPath + "history/" #training/modelNameAndTime/learningRate/FineTuning/LearningRate/history/
          makeDirs([fineTuningLRPath, weightsPath, plotsPath, historyPath])
          #freezo i layer se necessario
          if trainingInfo.getFreezeLayer() == True and trainingInfo.getFreezeFrom() - 1 >= iteration and trainingInfo.getLayersToFreeze >= 0:
            for layerIndex in range(trainingInfo.getLayersToFreeze + 1):
              model.layers[layerIndex].trainable = False
          #faccio il training
          tensorboardIterationPath = tensorboardPath + "FineTuning_" + str(learningRate) +  "_" + str(fineTuningLR) + "/"
          checkpointPath = '/tmp/epoch_{epoch:02d}_valLoss_{val_loss:.4f}.hdf5'
          #checkpointPath = weightsPath + "epoch_{epoch:02d}_valLoss_{val_loss:.4f}.hdf5"
          callbacks = [
              tensorboardCallback(logDir = tensorboardIterationPath,saveGraph = True, saveImages = True),
              earlyStoppingCallback(fineTuningEarlyStoppingDelta , fineTuningEarlyStoppinPatience ),
              checkpointCallback(fileName = checkpointPath, bestOnly = True, weightsOnly = True)
          ]
          history, weights = trainModel(
              model, fineTuningLR,
              trainingInfo.getTrainingData(), trainingInfo.getTrainingLabels(),
              trainingInfo.getValidationData(), trainingInfo.getValidationLabels(),
              trainingInfo.getFineTuningEpochs(), trainingInfo.getShuffle(), trainingInfo.getBatchSize(),
              lossFunction, metrics,
              callbacks = callbacks, weights = weights, verbose = verboseTraining)
          cleanCheckpointCallbackWeights('/tmp/')
          lastWeights = list(filter(lambda fn: fn.startswith('epoch') and fn.endswith('.hdf5'),os.listdir('/tmp/')))[0]
          shutil.move('/tmp/' + lastWeights, weightsPath + lastWeights)
          #cleanCheckpointCallbackWeights(weightsPath)
          saveLossPlot(plotsPath + "validationLoss.png", "Validation",[(history.history['val_loss'], learningRate)])
          saveLossPlot(plotsPath + "traininLoss.png", "Training",[(history.history['loss'], learningRate)])
          if 'val_sparse_categorical_accuracy' in history.history:
            saveAccuracyPlot(plotsPath + "validationAccuracy.png", "Validation",[(history.history['val_sparse_categorical_accuracy'],learningRate)])
          if 'sparse_categorical_accuracy' in history.history:
            saveAccuracyPlot(plotsPath + "trainingAccuracy.png", "Training",[(history.history['sparse_categorical_accuracy'],learningRate)])
          with open(historyPath + "history.history","wb") as pickleOut:
            pickle.dump(history.history, pickleOut)
    # fine ciclo sui learnin rate
  except Exception as e:
    print("Errore durante il training.\n")
    lastSession.pop()
    traceback.print_exc()
    if 'resumeTraining' in locals():
      if resumeTraining != None:
        print("Cleaning", modelPath)
        shutil.rmtree(modelPath)
    else:
      print("Cleaning", modelPath)
      shutil.rmtree(modelPath)
    
print("Fine training")